# The Multi-armed bandit

In [1]:
import tensorflow.compat.v1 as tf
import tf_slim as slim
import numpy as np
tf.compat.v1.disable_eager_execution()  # eager execution 관련 에러 해결

### The Bandit
- 손잡이가 4개인 밴딧
- pullBandit 함수 : 랜덤 표준정규분포 값 생성하고 인수로 받은 수보다 크면 1, 아니면 -1 반환

In [2]:
# 밴딧의 손잡이 목록 작성
# 현재 손잡이 4 (인덱스 #3)이 가장 자주 양의 보상 제공하도록 설정되어있음
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)
def pullBandit(bandit):
    # 랜덤한 표준정규분포값 생성
    result = np.random.randn(1)
    if result > bandit:
        # 양의 보상 반환
        return 1
    else:
        # 음의 보상 반환
        return -1

### The Agent
- 간단한 신경망 구현
- 각 손잡이에 대한 일련의 값들로 구성
- 각 값은 해당 뱃딧을 선택할 때 반환되는 보상의 추정값
- 정책 경사 방법을 이용해 큰 보상을 받도록 에이전트 업데이트

In [3]:
tf.reset_default_graph()

# 네트워크의 피드포워드 부분 구현
weights = tf.Variable(tf.ones([num_arms]))  # num_arms = 4
output = tf.nn.softmax(weights)  # 활성함수가 softmax인 신경망

# 학습 과정 구현
# 보상과 선택된 액션을 네트워크에 피드해줌으로써 비용(loss) 계산하고 
# 비용을 이용해 네트워크 업데이트

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(output, action_holder, [1])   #tf.slice(input_, begin,size)
loss = -(tf.log(responsible_output)*reward_holder)  # Loss = -log(pi)*A
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

### Training the Agent
- 이 환경에서 에이전트는 액션을 취함으로써 학습하고 보상을 받게 됨

In [4]:
total_episodes = 1000  # 에이전트를 학습시킬 총 에피소드 수
total_reward = np.zeros(num_arms)  # 밴딧 손잡이에 대한 점수판을 0으로 설정

# 그래프 실행 전 초기화를 해야 그 값이 변수에 지정됨
init = tf.global_variables_initializer() 

# 텐서플로 그래프 론칭
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:  # 에피소드 수만큼 반복 (1000회)
        # 볼츠만(Boltzmann) 분포에 따라 액션 선택
        actions = sess.run(output)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions==a)
        
        reward = pullBandit(bandit_arms[action])  # 손잡이 중 하나를 선택하여 보상을 받음
        
        # 네트워크 업데이트
        _, resp, ww = sess.run([update, responsible_output, weights], feed_dict={reward_holder:[reward], action_holder:[action]})
        
        # 보상의 총계 업데이트
        total_reward[action] += reward
        if i%50 == 0:
            print("Running reward for the " + str(num_arms) + "arms of the bandit: " + str(total_reward))
        i += 1
        
    print("\nThe agent thinks arm " + str(np.argmax(ww)+1) + " is the most promising....")
    if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
        print("...and it was right!")
    else:
        print("... and it was wrong!")

Running reward for the 4arms of the bandit: [1. 0. 0. 0.]
Running reward for the 4arms of the bandit: [-7. -1. -3.  8.]
Running reward for the 4arms of the bandit: [-14.  -1.   4.  20.]
Running reward for the 4arms of the bandit: [-17.   3.  14.  35.]
Running reward for the 4arms of the bandit: [-14.   7.  17.  49.]
Running reward for the 4arms of the bandit: [-13.   9.  28.  57.]
Running reward for the 4arms of the bandit: [-17.   8.  33.  67.]
Running reward for the 4arms of the bandit: [-22.   8.  35.  78.]
Running reward for the 4arms of the bandit: [-26.   6.  41.  86.]
Running reward for the 4arms of the bandit: [-27.   1.  42.  95.]
Running reward for the 4arms of the bandit: [-30.   8.  45. 104.]
Running reward for the 4arms of the bandit: [-33.   6.  49. 123.]
Running reward for the 4arms of the bandit: [-35.   9.  50. 135.]
Running reward for the 4arms of the bandit: [-35.   9.  51. 148.]
Running reward for the 4arms of the bandit: [-44.  10.  57. 156.]
Running reward for the